<a href="https://colab.research.google.com/github/imiro/bangkit-w05-winequality/blob/notebook-refactor/bangkit-w05-winequality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [2]:
#@title Import statements
#@content Tensorflow version

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

2.2.0-rc2


In [0]:
#@title (Misc) GDrive integration
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Read dataset from CSV file

In [0]:
url = 'https://raw.githubusercontent.com/imiro/bangkit-w05-winequality/master/datasets/winequality-red.csv'
df = pd.read_csv(url, error_bad_lines=False)

# Dataset characteristics

In [0]:
#@title Import plotting functions

import matplotlib.pyplot as plt
import seaborn as sns
import pandas.util.testing as tm
from collections import Counter

In [0]:
df.head(10)

Here we explore what our dataset has to offer.

In [0]:
df.info()

In [0]:
df.shape

In [0]:
df.columns

In [0]:
Counter(df['quality'])

In [0]:
sns.countplot(x='quality', data=df)

In [0]:
sns.pairplot(df)

In [0]:
qualityString = []
for i in df['quality'] :
  if 0 <= i < 5 :
    qualityString.append(0)
  elif 5 <= i < 7 :
    qualityString.append(1)
  else :
    qualityString.append(2)
df['Rating'] = qualityString
df['Rating2'] = df['quality'].copy()
df['Rating2'].map(lambda x: 0 if x < 5 else 1 if x < 7 else 2)

In [0]:
# df['Rating'] = df['Rating'].map({0: 'Bad', 1: 'Good'})
# df = pd.get_dummies(df, prefix='', prefix_sep='')
# df.tail()

In [0]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Rating
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023,1.096310
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569,0.407354
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000,1.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000,1.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,2.000000


# Data Prep

In [0]:
dataset = df.copy()
dataset.pop("quality")

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

## Create test data

We don't have test data available as is from the source, so we have to create it from the data pool.

We decided to train on 80% of the data, and make the rest 20% as the test data.

To avoid bias, we divide the train and test data with random sampling.

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [0]:
train_dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Rating
1109,10.8,0.470,0.43,2.10,0.171,27.0,66.0,0.99820,3.17,0.76,10.8,1
1032,8.1,0.820,0.00,4.10,0.095,5.0,14.0,0.99854,3.36,0.53,9.6,1
1002,9.1,0.290,0.33,2.05,0.063,13.0,27.0,0.99516,3.26,0.84,11.7,2
487,10.2,0.645,0.36,1.80,0.053,5.0,14.0,0.99820,3.17,0.42,10.0,1
979,12.2,0.450,0.49,1.40,0.075,3.0,6.0,0.99690,3.13,0.63,10.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
281,7.7,0.270,0.68,3.50,0.358,5.0,10.0,0.99720,3.25,1.08,9.9,2
932,7.6,0.400,0.29,1.90,0.078,29.0,66.0,0.99710,3.45,0.59,9.5,1
732,7.3,0.835,0.03,2.10,0.092,10.0,19.0,0.99660,3.39,0.47,9.6,1
715,7.2,0.490,0.18,2.70,0.069,13.0,34.0,0.99670,3.29,0.48,9.2,1


In [0]:
train_stats = train_dataset.describe()
train_stats.pop("Rating")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1279.0,8.310164,1.744190,4.60000,7.1000,7.90000,9.250000,15.9000
volatile acidity,1279.0,0.525571,0.176645,0.12000,0.3900,0.52000,0.635000,1.5800
citric acid,1279.0,0.271618,0.196822,0.00000,0.0900,0.26000,0.430000,0.7900
residual sugar,1279.0,2.516341,1.300985,0.90000,1.9000,2.20000,2.600000,13.9000
chlorides,1279.0,0.087347,0.045666,0.01200,0.0710,0.07900,0.091000,0.6110
free sulfur dioxide,1279.0,15.868647,10.438880,1.00000,7.0000,14.00000,21.000000,72.0000
total sulfur dioxide,1279.0,46.488663,32.952841,6.00000,22.0000,38.00000,62.000000,289.0000
density,1279.0,0.996739,0.001888,0.99007,0.9956,0.99676,0.997855,1.0032
pH,1279.0,3.312588,0.153923,2.87000,3.2100,3.31000,3.400000,4.0100
sulphates,1279.0,0.655012,0.160192,0.33000,0.5500,0.62000,0.730000,1.9800


## Separate label and features

In [0]:
train_labels = train_dataset.pop('Rating')
test_labels = test_dataset.pop('Rating')

## Normalize data

Earlier on descriptions of dataset characteristic, we see that range of the data varies.

For example, values of `fixed acidity` ranges from `4.6 - 15.9`, while `total sulfur dioxide` has a range of `6-62`. Meanwhile, `pH` values only range from `2.74` to `4.01`.

We fear that this can cause  issues when training the model. Therefore, we convert each value of the features to its Z-score.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [0]:
train_labels.shape

(1279,)

# Define function that build model

In [0]:
def build_model(my_learning_rate):
  model = keras.Sequential([
    layers.Flatten(input_shape=[(len(train_dataset.keys()))]),
    layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    layers.Dense(3),
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=my_learning_rate)

  model.compile(optimizer=optimizer,
              # loss='binary_crossentropy',
              # loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

In [0]:
# example_batch = normed_train_data[:10]
# example_result = model.predict(example_batch)
# example_result

# Set hyperparameters

In [0]:
EPOCHS = 100
learning_rate = 0.01

# Build and train the model

In [0]:
model = build_model(learning_rate)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 11)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1536      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 1,923
Trainable params: 1,923
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_path = "/content/gdrive/My Drive/Trained_Models/wine_binary_quality/wine_binary_quality.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(
  normed_train_data, train_labels,
  # train_dataset, train_labels,
  epochs=EPOCHS, 
  # validation_split = 0.2, 
  # verbose=0,
  # callbacks=[cp_callback])
  # callbacks=[tfdocs.modeling.EpochDots()])
  # batch_size=1
  )

Epoch 1/100
40/40 [==============================] - 0s 1ms/step - loss: 0.5498 - accuracy: 0.8233
Epoch 2/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4429 - accuracy: 0.8296
Epoch 3/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4382 - accuracy: 0.8350
Epoch 4/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4019 - accuracy: 0.8468
Epoch 5/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4092 - accuracy: 0.8436
Epoch 6/100
40/40 [==============================] - 0s 1ms/step - loss: 0.3898 - accuracy: 0.8460
Epoch 7/100
40/40 [==============================] - 0s 1ms/step - loss: 0.3941 - accuracy: 0.8468
Epoch 8/100
40/40 [==============================] - 0s 1ms/step - loss: 0.3961 - accuracy: 0.8475
Epoch 9/100
40/40 [==============================] - 0s 1ms/step - loss: 0.3804 - accuracy: 0.8436
Epoch 10/100
40/40 [==============================] - 0s 2ms/step - loss: 0.3827 - accuracy: 0.8483
Epoch 11/

In [0]:
test_loss, test_acc = model.evaluate(normed_test_data, test_labels, verbose=2)
# test_loss, test_acc = model.evaluate(test_dataset, test_labels, verbose=2)
print('Test accuracy:', test_acc)

10/10 - 0s - loss: 1.1818 - accuracy: 0.8656
Test accuracy: 0.8656250238418579


In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [0]:
predictions = probability_model.predict(
    normed_test_data)
# predictions

In [0]:
test_labels

In [0]:
i = len(predictions)
np.argmax(predictions[i-1])

1

# (Misc) Export model(s) to GDrive

In [0]:
checkpoint_path = "/content/gdrive/My Drive/Trained_Models/wine_binary_classifier/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [0]:
model.save_weights('/content/gdrive/My Drive/Trained_Models/wine_binary_classifier/wine_binary_quality_model')

In [0]:
model_save_name = 'wine_binary_classifier'
save_model_path = '/content/gdrive/My Drive/Trained_Models/wine_binary_classifier'

In [0]:
model.save(save_model_path)

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Trained_Models/wine_binary_classifier/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Trained_Models/wine_binary_classifier/assets


In [0]:
model.save('/content/gdrive/My Drive/Trained_Models/wine_binary_classifier/wine_binary_classifier.h5') 

In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_model_path)
tflite_model = converter.convert()
open("/content/gdrive/My Drive/Trained_Models/wine_binary_classifier/wine_binary_classifier.tflite", "wb").write(tflite_model)

8776